In [4]:
import numpy as np 
import pandas as pd
import keras
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Input, concatenate, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras import optimizers
from keras.utils import to_categorical
import os

from keras.layers import Bidirectional

Using TensorFlow backend.


In [5]:
import os
os.chdir("F:/talentsprintproject/quora-insincere-questions-classification")


In [6]:
import re

In [7]:
MAX_TIME = 30
VOCAB_SIZE = 20000
QUES_CLEANING_PATTERN = re.compile("[\s\n\r\t.,:;\-_\'\"?!#&()\/%\[\]\{\}\<\>\\$@\!\*\+\=]")
LSTM_DIM = 256
LSTM_DIMS = [512,256]
NUM_FILTERS = 5
FILTER_LENGTHS = [1,2,3,4,5,8,10,15,20,25]
DROPOUT = 0.4
LEARNING_RATE = 0.005
NUM_EPOCHS = 10
BATCH_SIZE = 2000
NUM_UNDERSAMPLE = 3
RUS_RATIO = 1.0/4.0

In [8]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [9]:
train_data = np.array(train_data)
test_data = np.array(test_data)

In [10]:
ct_0 = 0
c = 0
for d in train_data:
    if d[2] == 0:
        ct_0 += 1
    if len(QUES_CLEANING_PATTERN.split(d[1])) > MAX_TIME:
        c+=1
print(ct_0, len(train_data) - ct_0, c/len(train_data))

1225312 80810 0.05135890827962472


In [11]:
NUM_TRAIN = int(len(train_data)*0.8)
np.random.shuffle(train_data)
train_x = train_data[:NUM_TRAIN,1]
train_y = train_data[:NUM_TRAIN,2]
val_x = train_data[NUM_TRAIN:,1]
val_y = train_data[NUM_TRAIN:,2]
del train_data


In [12]:
test_x = test_data[:,1]
test_ids = test_data[:,0]
del test_data

In [13]:
tokenizer = Tokenizer(VOCAB_SIZE)
tokenizer.fit_on_texts(train_x)

In [14]:
train_x = tokenizer.texts_to_sequences(train_x)
val_x = tokenizer.texts_to_sequences(val_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [15]:
train_y.shape

(1044897,)

In [16]:
def getEmbeddingMatrix(wordIndex):
    
    embeddingsIndex = {}
    # Load the embedding vectors from ther GloVe file
    with open("embeddings/glove.840B.300d/glove.840B.300d.txt", encoding="utf8") as f:
        for line in f:
            values = line.split(' ')
            word = values[0]
            try:
                embeddingVector = np.asarray(values[1:], dtype='float32')
            except:
                print(values)
                break
            embeddingsIndex[word] = embeddingVector

    print('Found %s word vectors.' % len(embeddingsIndex))

    # Minimum word index of any word is 1.
    embeddingMatrix = np.zeros((len(wordIndex) + 1, 300))
    for word, i in wordIndex.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector
    del embeddingsIndex
    return embeddingMatrix

In [17]:
wordIndex = tokenizer.word_index
wI = {}
for k,v in wordIndex.items():
    if v < VOCAB_SIZE:
        wI[k] = v
wordIndex = wI

In [18]:
embeddingMatrix = getEmbeddingMatrix(wordIndex)

Found 2196016 word vectors.


In [19]:
train_x = pad_sequences(train_x,maxlen=MAX_TIME,padding='post')
val_x = pad_sequences(val_x,maxlen=MAX_TIME,padding='post')
test_x = pad_sequences(test_x,maxlen=MAX_TIME,padding='post')

In [20]:
train_y = to_categorical(train_y)
#train_y = train_y.astype('int')
val_y = to_categorical(val_y)

In [21]:
train_y.shape

(1044897, 2)

In [22]:
def buildModel(embeddingMatrix):
    embeddingLayer = Embedding(embeddingMatrix.shape[0],300,weights=[embeddingMatrix],\
                              input_length=MAX_TIME,trainable=False)
   
    sent_inp = Input(shape=(MAX_TIME,))
    sent = embeddingLayer(sent_inp)
    
    _, h1, c1 = LSTM(LSTM_DIM,dropout=DROPOUT,return_state=True)(sent)
    h1 = Reshape([MAX_TIME,LSTM_DIM])(concatenate([h1]*MAX_TIME,1))
    lstm_inp = concatenate([sent,h1],axis=2)
    _, h2, c2 = LSTM(LSTM_DIM*2,dropout=DROPOUT,return_state=True)(lstm_inp)
    probs = Dense(64,activation='relu')(h2)
    probs = Dense(2,activation='sigmoid')(probs)
    model = Model(inputs=sent_inp,outputs=probs)
    rmsprop = optimizers.rmsprop(lr=LEARNING_RATE)
    
    model.compile(loss='categorical_crossentropy',
                 optimizer=rmsprop,
                 metrics=['acc'])
    return model

In [24]:
model = buildModel(embeddingMatrix)

In [ ]:
model.fit(train_x,train_y,epochs=NUM_EPOCHS,batch_size=BATCH_SIZE,verbose=1,validation_data=(val_x,val_y))

Train on 1044897 samples, validate on 261225 samples
Epoch 1/10
1044897/1044897 [==============================] - 6814s 7ms/step - loss: 0.1743 - acc: 0.9426 - val_loss: 0.1214 - val_acc: 0.9525
Epoch 2/10
1044897/1044897 [==============================] - 7843s 8ms/step - loss: 0.1220 - acc: 0.9523 - val_loss: 0.1285 - val_acc: 0.9530
Epoch 3/10
 150000/1044897 [===>..........................] - ETA: 1:27:20 - loss: 0.1159 - acc: 0.9546

In [ ]:
predictions = model.predict(test_x,batch_size=BATCH_SIZE)
predictions = predictions.argmax(axis=1)